In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import obspy

from IPython.display import Image

In [ ]:
def read_trace(filename):
    """Reads an ASCII file and returns a obspy Traces"""
    data = np.loadtxt(filename)
    # first column is time, second column is the data
    times = data[:, 0]
    disp = data[:, 1]
    # get station name from the filename
    net, sta, comp, *_ = filename.split("/")[-1].split(".")
    delta = times[1] - times[0]
    headers = {"station": sta, "network": net, "channel": comp, "delta": delta, "b": times[0]}
    return obspy.Trace(disp, headers)

# Running specfem2d workflow with Seisflows

> **SeisFlows** is an open-source, Python-based waveform inversion package meant to automate the problems of full waveform inversion, seismic migration, and adjoint tomography on high performance computers.

[Documentation](https://seisflows.readthedocs.io/en/latest/index.html)

Setting the relative paths by running [./setup_paths.sh](./setup_paths.sh)

In [ ]:
! ./setup_paths.sh

In [ ]:
# clean directory
! seisflows clean -f

We can check [parameters.yaml](parameters.yaml)

In [ ]:
# stop the workflow after gradient computation
! seisflows par stop_after evaluate_gradient_from_kernels

In [ ]:
# run the workflow
! seisflows submit

In [ ]:
# Plot the 'true' or target model
! seisflows plot2d MODEL_TRUE vs -s "model_true.png"
Image("model_true.png")

In [ ]:
! seisflows plot2d GRADIENT_01 vs_kernel -s "kernel.png"
Image("kernel.png")

In [ ]:
# plot a record
! seisflows plotst ./scratch/solver/01/traces/syn/AA.A0000.BXY.semd --save trace.png
Image(filename="trace.png")

In [ ]:
obs = read_trace("./scratch/solver/01/traces/obs/AA.A0000.BXY.semd")
syn = read_trace("./scratch/solver/01/traces/syn/AA.A0000.BXY.semd")
adj = read_trace("./scratch/solver/01/traces/adj/AA.A0000.BXY.adj")

In [ ]:
fig, (ax, bx) = plt.subplots(figsize=(18, 6), nrows=2)
ax.plot(obs.times(), obs.data, "k")
ax.plot(syn.times(), syn.data, "r")
ax.set_xlim(0, 280)
bx.plot(adj.times(), adj.data, "g")

In [ ]:
! seisflows par stop_after null
# Resume the workflow
! seisflows submit

In [ ]:
! seisflows plot2d MODEL_01 vs -s "model_01.png"
Image("model_01.png")

In [ ]:
! seisflows plot2d MODEL_02 vs -s "model_02.png"
Image("model_02.png")